In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
import scipy  #Used to upsample our image
import tensorflow

import os
import cv2
from PIL import Image
import numpy as np



np.random.seed(43)
tensorflow.random.set_seed(43)

#Read images and get them ready for training

path = r'C:\Users\user\OneDrive\Desktop\data\MURA-v1.1\train'
SIZE = 224

import os
dataset = [] 

images_array=[]
anomaly = []
part_label=[]
y_train_class_0=[]
y_train_class_1=[]
y_train_class_2=[]


images_per_category = 1000
image_counter = {part: 0 for part in ['XR_HUMERUS', 'XR_WRIST', 'XR_ELBOW']}


for body in os.listdir(path):
    body_part = body
    if body_part in ['XR_HUMERUS', 'XR_WRIST', 'XR_ELBOW']:  # Only include specific body parts
        path_p = os.path.join(path, body_part)
        count = 0  # Counter to keep track of images per category
        
        for id_p in os.listdir(path_p):
            if count >= images_per_category:  
                break
            patient_id = id_p
            path_id = os.path.join(path_p, id_p)
            for lab in os.listdir(path_id):
                labb = lab
                if count >= images_per_category:  
                    break
                if lab.split('_')[-1] == 'positive':
                    label = 1
                elif lab.split('_')[-1] == 'negative':
                    label = 0
                path_l = os.path.join(path_id, labb)
                for img in os.listdir(path_l):
                    if count >=images_per_category:  # Check if we already have 10 images from this category
                        break
                    img_path = os.path.join(path_l, img)
                    image = cv2.imread(img_path)
                    if image is None:
                        print(f"Failed to read image: {img_path}")
                    else:
                        image = Image.fromarray(image, 'RGB')
                        image = image.resize((SIZE, SIZE))
                        images_array.append(np.array(image))
                        anomaly.append(label)
                        part_label.append(body_part)
                        if body_part == 'XR_HUMERUS':
                            y_train_class_0.append(1)
                            y_train_class_1.append(0)
                            y_train_class_2.append(0)
                        elif body_part == 'XR_WRIST':
                            y_train_class_0.append(0)
                            y_train_class_1.append(1)
                            y_train_class_2.append(0)
                        elif body_part == 'XR_ELBOW':
                            y_train_class_0.append(0)
                            y_train_class_1.append(0)
                            y_train_class_2.append(1)
                        count += 1  # Increment counter after adding an image
                        
y_train_class_0=np.array(y_train_class_0)
y_train_class_0
y_train_class_1=np.array(y_train_class_1)
y_train_class_1
y_train_class_2=np.array(y_train_class_2)
y_train_class_2

array([1, 1, 1, ..., 0, 0, 0])

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
import scipy  #Used to upsample our image
import os
import cv2
from PIL import Image
import numpy as np


images_array = np.array(images_array)
anomaly = np.array(anomaly)

#Split into train and test data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images_array,y_train_class_1, test_size = 0.20, random_state = 42)

#Without scaling (normalize) the training may not converge. 
#so that all values are within the range of 0 and 1.

X_train = X_train /255.
X_test = X_test /255.

 
from tensorflow.keras.utils import to_categorical
pp=y_train
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [3]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam # Corrected import
import numpy as np

# Define the model
def get_model(input_shape=(224, 224, 3)):
    resenet = ResNet50(weights=None, include_top=False, input_shape=input_shape)


    x = resenet.output
    x = Flatten()(x) # Added parentheses
    x = Dense(32, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)  # Changed activation function to sigmoid

    model = Model(resenet.input, x)
    model.compile(loss="binary_crossentropy", 
                  optimizer=Adam(learning_rate=0.01), 
                  metrics=["accuracy"])
    
    return model

model = get_model(input_shape=(224, 224, 3))
print(model.summary())




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [4]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(X_test, y_test))

Epoch 1/50


75/75 [==============================] - 666s 9s/step - loss: 9.9049 - accuracy: 0.6237 - val_loss: 0.6598 - val_accuracy: 0.6900
Epoch 2/50
75/75 [==============================] - 626s 8s/step - loss: 0.6532 - accuracy: 0.6608 - val_loss: 0.6312 - val_accuracy: 0.6900
Epoch 3/50
75/75 [==============================] - 625s 8s/step - loss: 0.6383 - accuracy: 0.6608 - val_loss: 0.6039 - val_accuracy: 0.6900
Epoch 4/50
75/75 [==============================] - 623s 8s/step - loss: 0.6313 - accuracy: 0.6604 - val_loss: 0.6235 - val_accuracy: 0.6900
Epoch 5/50
75/75 [==============================] - 622s 8s/step - loss: 0.6322 - accuracy: 0.6608 - val_loss: 0.6446 - val_accuracy: 0.6900
Epoch 6/50
75/75 [==============================] - 625s 8s/step - loss: 0.6055 - accuracy: 0.6608 - val_loss: 0.6703 - val_accuracy: 0.6900
Epoch 7/50
75/75 [==============================] - 622s 8s/step - loss: 0.7256 - accuracy: 0.6608 - val_loss: 0.5814 - val_accuracy: 0.6900
Epoch 8/50


In [5]:
# Save the model
model.save('model_wrist.h5')

# Save the training history
import pickle

with open('model_wrist_training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
